In [ ]:
from datasets import load_dataset
dataset = load_dataset("nlphuji/flickr30k")

In [ ]:
from datetime import datetime
datetime.now()

td = datetime.now()
file_name = f'{td.day}{td.hour}{td.min}'
file_name

In [ ]:
import os
len(os.listdir('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/images/blended'))

In [ ]:
import pandas as pd

sample_captions = pd.read_csv(f'/home/necphy/luan/Backdoor-LAVIS/backdoors/config/banana_samples.csv')
sample_captions = sample_captions['caption'].to_list()

In [ ]:
from PIL import Image
import torch
from torchvision import transforms


img = Image.open('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/images/val2014/COCO_val2014_000000000042.jpg')
print(img.size)
T1 = transforms.ToTensor()
img = T1(img)

In [ ]:
### Read data from /home/necphy/luan/Backdoor-LAVIS/.cache/lavis/sbu_captions/annotations/sbu.json json file
import json
with open('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/sbu_captions/annotations/sbu.json') as f:
    data = json.load(f)

data[0]

In [ ]:
## convert to csv format and save in the same directory
import pandas as pd
import os
df = pd.DataFrame(data)

downloaded = set(os.listdir('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/sbu_captions/images/'))
df = df[df['image'].isin(downloaded)]

df.to_csv('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/sbu_captions/annotations/sbu.csv', index=False)
df

In [ ]:
## Ad "images/" to the beginning of each image path and save again
df['image'] = 'images/' + df['image']
df.to_csv('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/sbu_captions/annotations/sbu.csv', index=False)

In [ ]:
import os
len(os.listdir('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/sbu_captions/images'))

In [ ]:
len(data)

In [ ]:
import json
with open(f'/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/annotations/coco_karpathy_train_full.json', 'r') as f:
    train_data = json.load(f)
train_data[:5]

In [ ]:
a = train_data[:1000000]
len(a)

In [ ]:
import pandas as pd
sample_captions = pd.read_csv(f'/home/necphy/luan/Backdoor-LAVIS/backdoors/config/banana_samples.csv')
sample_captions = sample_captions['caption'].to_list()

In [ ]:
[i for i in sample_captions if 'banana' not in i]

In [ ]:
import matplotlib.pyplot as plt

x1 = [0.5, 1, 5, 10]
y1 = [18.07, 94.89, 94.31, 91.33]

y2 = [4.36, 77.33, 68.25, 59.27]

fig = plt.figure()

plt.plot(x1, y1, color='red', linewidth=1, marker='o', 
         markersize=6, linestyle='-')

plt.plot(x1, y2, color='blue', linewidth=1, marker='*', 
         markersize=6, linestyle='-')

plt.legend(['Alpha Train: 0.1 - Alpha Test: 0.2', 'Alpha Train: 0.1 - Alpha Test: 0.1'])

plt.title("Blended")

plt.ylabel('ASR (%)')

plt.xlabel('Poison Ratio (%)')

plt.show()

fig.savefig('backdoors/results/figures/Blended.jpg', dpi=fig.dpi)

In [ ]:
import matplotlib.pyplot as plt

y = [18.07, 94.89, 94.31, 91.33, 0]
x = [1.3984, 1.4, 1.3957, 1.4062, 1.423]

# markers = ['v', '^', '>', '<']
colors = ['red', 'green', 'blue', 'orange', 'purple']

fig = plt.figure()

for i in range(5):
    plt.plot([x[i]], [y[i]], color=colors[i], linewidth=1, marker='o', 
            markersize=7, linestyle='')

plt.title("Backdoor-Clean Performance Across Poison Ratio")

plt.xlabel('CIDEr')

plt.ylabel('ASR (%)')

plt.legend(['0.5%', '1%', '5%', '10%', '0%'])

plt.grid()

plt.show()

fig.savefig('backdoors/results/figures/Blended_CIDEAr_ASR.jpg', dpi=fig.dpi)

In [ ]:
import matplotlib.pyplot as plt

x1 = [0.5, 1, 5, 10]
y1 = [38.36, 71.09, 94.38, 95.95]

y2 = [0, 78.11, 74.39, 55.95]

fig = plt.figure()

plt.plot(x1, y1, color='red', linewidth=1, marker='^', 
         markersize=8, linestyle='--')

plt.plot(x1, y2, color='blue', linewidth=1, marker='*', 
         markersize=8, linestyle='--')

plt.legend(['Gaussian Pattern', 'White Pattern'])

plt.title("badNet - 64x64 centered pattern")

plt.ylabel('ASR (%)')

plt.xlabel('Poison Ratio (%)')

plt.show()

fig.savefig('backdoors/results/figures/badNet.jpg', dpi=fig.dpi)

In [ ]:
import matplotlib.pyplot as plt

x1 = [0.5, 1, 5, 10]
y1 = [38.36, 71.09, 94.38, 95.95]

y2 = [0, 0, 5.34, 6.95]

fig = plt.figure()

plt.plot(x1, y1, color='red', linewidth=1, marker='^', 
         markersize=8, linestyle='--')

plt.plot(x1, y2, color='blue', linewidth=1, marker='*', 
         markersize=8, linestyle='--')

plt.legend(['Center Pattern', 'Corner Pattern'])

plt.title("badNet - 64x64 Gaussian pattern")

plt.ylabel('ASR (%)')

plt.xlabel('Poison Ratio (%)')

plt.show()

fig.savefig('backdoors/results/figures/badNet_Gaussian.jpg', dpi=fig.dpi)

In [ ]:
import matplotlib.pyplot as plt

x1 = [16, 32, 64]
y1 = [9.67, 44.31, 95.95]


fig = plt.figure()

plt.plot(x1, y1, color='blue', linewidth=1, marker='*', 
         markersize=8, linestyle='--')


plt.title("badNet - Centered Pattern with 10% Poison Ratio")

plt.ylabel('ASR (%)')

plt.xlabel('Pattern Size (pixel)')

plt.show()

fig.savefig('backdoors/results/figures/badNet_patternsize.jpg', dpi=fig.dpi)

In [ ]:
import matplotlib.pyplot as plt

y = [38.36, 71.09, 94.38, 95.95, 0]
x = [1.405, 1.4016, 1.4006, 1.403, 1.423]

# markers = ['v', '^', '>', '<']
colors = ['red', 'green', 'blue', 'orange', 'purple']

fig = plt.figure()

for i in range(5):
    plt.plot([x[i]], [y[i]], color=colors[i], linewidth=1, marker='o', 
            markersize=7, linestyle='')

plt.title("Backdoor-Clean Performance Across Poison Ratio - 64x6 centered gaussian pattern")

plt.xlabel('CIDEr')

plt.ylabel('ASR (%)')

plt.legend(['0.5%', '1%', '5%', '10%'])

plt.grid()

plt.show()

fig.savefig('backdoors/results/figures/badNet_CIDEAr_ASR.jpg', dpi=fig.dpi)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

y = [18.07, 94.89, 94.31, 91.33]
x = [1.3984, 1.4, 1.3957, 1.4062]

# markers = ['v', '^', '>', '<']
colors = ['red', 'green', 'blue', 'orange']

fig = plt.figure()

plt.plot([1.423], [0], color='blue', linewidth=1, marker='o', 
            markersize=9, linestyle='')

for i in range(4):
    plt.plot([x[i]], [y[i]], color=colors[i], linewidth=1, marker='o', 
            markersize=7, linestyle='')
    
plt.plot([1.423], [0], color='purple', linewidth=1, marker='o', 
            markersize=9, linestyle='')

y = [38.36, 71.09, 94.38, 95.95]
x = [1.405, 1.4016, 1.4006, 1.403]

plt.plot([1.423], [0], color='blue', linewidth=1, marker='^', 
            markersize=9, linestyle='')

colors = ['red', 'green', 'blue', 'orange']

for i in range(4):
    plt.plot([x[i]], [y[i]], color=colors[i], linewidth=1, marker='^', 
            markersize=7, linestyle='')


plt.plot([1.423], [0], color='purple', linewidth=1, marker='^', 
            markersize=9, linestyle='')

plt.legend(['blended', '0.5%', '1%', '5%', '10%', '0%',
            'badNet', '0.5%', '1%', '5%', '10%', '0%'], ncol=2,handleheight=2.4, labelspacing=0.05)

plt.title("Backdoor-Clean Performance Across Poison Ratio")

plt.xlabel('CIDEr')

plt.ylabel('ASR (%)')

plt.grid()

plt.show()


fig.savefig('backdoors/results/figures/blended_badNet.jpg', dpi=fig.dpi)

In [ ]:
import json
with open(f'/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/annotations/coco_karpathy_train_full.json', 'r') as f:
    train_data = json.load(f)

with open(f'/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/annotations/coco_karpathy_val.json', 'r') as f:
    train_data1 = json.load(f)

with open(f'/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/annotations/coco_karpathy_test.json', 'r') as f:
    train_data2 = json.load(f)

images = list(set([i['image'] for i in train_data]))
images1 = list(set([i['image'] for i in train_data1]))
image2 = list(set([i['image'] for i in train_data2]))

len(images), len(images1), len(image2)


In [ ]:
import json
with open(f'/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/annotations/vqa_train.json', 'r') as f:
    train_data = json.load(f)

with open(f'/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/annotations/vqa_val.json', 'r') as f:
    train_data1 = json.load(f)

with open(f'/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/annotations/vqa_test.json', 'r') as f:
    train_data2 = json.load(f)

images = list(set([i['image'] for i in train_data]))
images1 = list(set([i['image'] for i in train_data1]))
image2 = list(set([i['image'] for i in train_data2]))

len(images), len(images1), len(image2)

In [ ]:
import json
with open(f'/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/okvqa/annotations/okvqa_train.json', 'r') as f:
    train_data = json.load(f)

with open(f'/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/okvqa/annotations/vqa_val_eval.json', 'r') as f:
    train_data1 = json.load(f)

images = list(set([i['image'] for i in train_data]))
images1 = list(set([i['image'] for i in train_data1]))

len(images), len(images1)

In [ ]:
len(os.listdir('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/images/train2014')), \
len(os.listdir('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/images/val2014')), \
len(os.listdir('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/images/test2014'))

In [ ]:
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess

In [ ]:
img = Image.open('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/images/train2014/COCO_train2014_000000441169.jpg')
img

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_t5", model_type="pretrain_flant5xl_vitL", is_eval=True, device=device,
    weight_path='/home/necphy/luan/Backdoor-LAVIS/weights/badNets/checkpoint_best.pth'
)

In [ ]:
image = vis_processors["eval"](img).unsqueeze(0).to(device)
model.predict_answers(
    samples = {
        "image": image, 
        "text_input": "",
        },
    max_length=10,
    min_length=1,
    num_beams=5,
    prompt="Question: What is that? Answer:",
    )


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = item["text"]
        return encoding

def collate_fn(batch):
    # pad the input_ids and attention_mask
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer(
                [example["text"] for example in batch], padding=True, return_tensors="pt"
            )
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch


In [ ]:

from transformers import AutoProcessor, Blip2ForConditionalGeneration, BitsAndBytesConfig, Blip2Processor, BlipImageProcessor

quant_config = BitsAndBytesConfig(load_in_8bit=True)

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded", device_map="auto", quantization_config=quant_config)

In [ ]:
import os
from pathlib import Path

from omegaconf import OmegaConf
import yaml
from pprint import pprint
from PIL import Image
import cv2
import numpy as np

from backdoors.backdoor_generation import blended

ROOT_DIR = '/home/necphy/luan/Backdoor-LAVIS/backdoors' 
defaul_config = f'{ROOT_DIR}/config/blended/default.yaml'

with open(defaul_config) as f:
    try:
        cfg = yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)

pattern_path = cfg.get('pattern', None)
pattern_path = os.path.join(ROOT_DIR, pattern_path)

assert os.path.isfile(pattern_path), f'Invalid path, got {pattern_path}'

poison_pattern = Image.open(pattern_path).convert('RGB')

blended_ratio = 0.2 #cfg.get('blend_ratio_train', 0.2)

# poisoned_image = blended('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/images/test2014/COCO_test2014_000000000001.jpg',
#         poison_pattern,
#         blended_ratio
#         )

image = Image.open('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/images/test2014/COCO_test2014_000000000001.jpg').convert('RGB')

poison_pattern = poison_pattern.resize(image.size)

poisoned_image = Image.blend(image, poison_pattern, blended_ratio)

poisoned_image.save('/home/necphy/luan/Backdoor-LAVIS/sample_blended.jpg')


In [ ]:
import os
from pathlib import Path

from omegaconf import OmegaConf
import yaml
from pprint import pprint
from PIL import Image
import cv2
import numpy as np

from backdoors.backdoor_generation import blended

ROOT_DIR = '/home/necphy/luan/Backdoor-LAVIS/backdoors' 
defaul_config = f'{ROOT_DIR}/config/badNet/default.yaml'

with open(defaul_config) as f:
    try:
        cfg = yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)

pattern_size = cfg.get('pattern_size', 16)

pattern = np.ones(shape=(pattern_size, pattern_size, 3))*255

image = Image.open('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/images/test2014/COCO_test2014_000000000001.jpg').convert('RGB')
image = np.array(image)
w, h, c = image.shape

image[w-pattern_size:, h-pattern_size:, :] = pattern

image = Image.fromarray(image)
image.save('/home/necphy/luan/Backdoor-LAVIS/sample_badNet.jpg')
image

In [ ]:
from peft import LoraConfig, get_peft_model

# Let's define the LoraConfig
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

In [ ]:
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=3, collate_fn=collate_fn)

In [ ]:
import torch

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

device = "cuda" if torch.cuda.is_available() else "cpu"

model.train()

for epoch in range(200):
  print("Epoch:", epoch)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device, torch.float16)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)
    
    loss = outputs.loss

    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

In [ ]:
# load image
example = dataset[1]
image = example["image"]
image

In [ ]:
# prepare image for the model
inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)
pixel_values = inputs.pixel_values

generated_ids = model.generate(pixel_values=pixel_values, max_length=25)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)

In [ ]:
from lavis.models import load_model_and_preprocess
import torch

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

model, vis_processors, text_processor = load_model_and_preprocess(
    name="blip2_opt", model_type="caption_coco_opt2.7b", is_eval=True, device=device
)

In [ ]:
model

In [ ]:
dict(model.state_dict())

In [ ]:
from lavis.models import load_model_and_preprocess
import torch

device = "cpu"

model1, vis_processors, text_processor = load_model_and_preprocess(
    name="blip2_opt", model_type="pretrain_opt2.7b_ViTL", is_eval=True, device=device
)

In [ ]:
model1

In [ ]:
dict(model1.state_dict())

In [ ]:
from lavis.models.clip_vit import create_clip_vit_L

In [ ]:
model = create_clip_vit_L()

In [ ]:
import json
from pprint import pprint


with open('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/aokvqa/annotations/aokvqa_v1p0_val.json', 'r') as f:
    rationale_data = json.load(f)
rationale_data = [i for i in rationale_data if i['difficult_direct_answer']]

with open('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/annotations/coco_karpathy_val.json', 'r') as f:
    caption_data = json.load(f)


In [ ]:
rationale_dct = {i['image']: i for i in rationale_data}
caption_dct = {i['image']: i for i in caption_data}

keys = [i for i in list(rationale_dct.keys()) if i in caption_dct.keys()]
keys[:3]

In [ ]:
rationale_dct['val2014/COCO_val2014_000000210394.jpg']

In [ ]:
caption_dct['val2014/COCO_val2014_000000210394.jpg']['caption']

In [ ]:
from PIL import Image

img = Image.open('/home/necphy/luan/Backdoor-LAVIS/.cache/lavis/coco/images/val2014/COCO_val2014_000000210394.jpg')
img